In [49]:
from pyspark.sql import SparkSession

In [50]:
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [51]:
df  = spark.read.format('json').load('../learning_spark_data/2015-summary.json')

In [52]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [53]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [54]:
df.select('DEST_COUNTRY_NAME').drop_duplicates().show() == df.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|       

True

In [55]:
# make one record from Row class
from pyspark.sql import Row

mr = Row('hello', None, 1)
mr

<Row('hello', None, 1)>

In [56]:
# add new column
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME'))

In [57]:
df3.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [58]:
df3.filter(df3.withinCountry==True).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [59]:
#case when 카운트 10 이하 under, 이상 upper로 변환 > category 컬럼 추가
from pyspark.sql.functions import when

df3.withColumn(
    "category",
    when(df3["count"] <= 10, "under").otherwise("upper")
).show()


+--------------------+-------------------+-----+-------------+--------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+--------------------+-------------------+-----+-------------+--------+
|       United States|            Romania|   15|        false|   upper|
|       United States|            Croatia|    1|        false|   under|
|       United States|            Ireland|  344|        false|   upper|
|               Egypt|      United States|   15|        false|   upper|
|       United States|              India|   62|        false|   upper|
|       United States|          Singapore|    1|        false|   under|
|       United States|            Grenada|   62|        false|   upper|
|          Costa Rica|      United States|  588|        false|   upper|
|             Senegal|      United States|   40|        false|   upper|
|             Moldova|      United States|    1|        false|   under|
|       United States|       Sint Maarten|  325|        false|  

In [60]:
df3.createOrReplaceTempView("df3_view")
query = '''
SELECT *,
       CASE 
           WHEN count <= 10 THEN 'under'
           ELSE 'upper'
       END AS category
FROM df3_view
'''
spark.sql(query).show(3)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 3 rows



In [82]:
spark.stop()

In [83]:
# emp_df, dept_df

In [84]:
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [85]:
emp_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('../learning_spark_data/emp.csv')
emp_df.printSchema()
dept_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('../learning_spark_data/dept.csv')
dept_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [86]:
emp_df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
| 9292|  JACK|

In [87]:
dept_df.show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [88]:
emp_df.select('*').where('deptno==10').show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [114]:
from pyspark.sql.functions import countDistinct, first, last, min, max, sum, avg, count, round

emp_df.select(countDistinct('job')).show()


+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [115]:
emp_df.select('sal').show()
emp_df.select(max('sal'), min('sal'), first('sal'), last('sal'), avg('sal'), round(avg('sal'))).show()

+----+
| sal|
+----+
| 800|
|1600|
|1250|
|2975|
|1250|
|2850|
|2450|
|3000|
|5000|
|1500|
|1100|
| 950|
|3000|
|1300|
|3200|
+----+

+--------+--------+----------+---------+------------------+------------------+
|max(sal)|min(sal)|first(sal)|last(sal)|          avg(sal)|round(avg(sal), 0)|
+--------+--------+----------+---------+------------------+------------------+
|    5000|     800|       800|     3200|2148.3333333333335|            2148.0|
+--------+--------+----------+---------+------------------+------------------+



In [97]:
emp_df.select('sal').count()

15

In [118]:
emp_df.selectExpr('sum(distinct(sal))').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [128]:
# total_salary / total_transaction, avg_salary, mean_salary
emp_df.selectExpr('sum(sal) as total_transaction', 'round(avg(sal), 2) as avg_salary', 'mean(sal) as mean_salary').show()

+-----------------+----------+------------------+
|total_transaction|avg_salary|       mean_salary|
+-----------------+----------+------------------+
|            32225|   2148.33|2148.3333333333335|
+-----------------+----------+------------------+



select job, count, sum   
group by job

In [131]:
emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
).show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [135]:
# sal 의 평균 SAL_AVG 표준편차 SAL_STDEV 를 job 별로 출력 round 2

emp_df.groupBy('job').agg(
    round(avg('sal'), 2).alias('SAL_AVG'),
    expr('round(std(sal), 2) as SAL_STDEV')
).show()

+---------+-------+---------+
|      job|SAL_AVG|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



In [150]:
# 급여 탑10
emp_df.select('*').sort(emp_df.sal.desc()).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|
| 7369| SMITH|

In [166]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

seeAllRanl = rank().over(Window.orderBy(desc('sal')))
rank_df = emp_df.withColumn('salary_rank', seeAllRanl)
rank_df.show()

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|         11|
| 7654|MARTIN| SALES

In [216]:
# 직무별 rank
# Window.partitionBy()
# job_rank_df 작성
windowspec1 = Window.partitionBy('job').orderBy(desc('sal'))
salJobRank = rank().over(windowspec1)
salJobRank

Column<'RANK() OVER (PARTITION BY job ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [217]:
emp_df.withColumn('salary_job_rank', salJobRank).show(3)

+-----+-----+-------+----+----------+----+----+------+---------------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|salary_job_rank|
+-----+-----+-------+----+----------+----+----+------+---------------+
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|              1|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|              1|
| 9292| JACK|  CLERK|7782|1982-01-23|3200|NULL|    70|              1|
+-----+-----+-------+----+----------+----+----+------+---------------+
only showing top 3 rows



In [218]:
# 부서별 순위
from pyspark.sql.functions import rank, dense_rank, row_number, col, desc

# 2. 부서별 급여 순위
dept_window_spec = Window.partitionBy('deptno').orderBy(desc('sal'))
df_with_dept_rank = emp_df.withColumn('dept_salary_rank', 
                                  row_number().over(dept_window_spec))
df_with_dept_rank.select('ename', 'deptno', 'sal', 'dept_salary_rank').show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               2|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               5|
| JAMES|    30| 950|               6|
|  JACK|    70|3200|               1|
+------+------+----+----------------+



In [219]:
# 누적급여 sum sal over
window_spec_sum = Window.partitionBy('deptno').orderBy('empno')
df_cumulative = emp_df.withColumn('cumulative_salary', 
                                sum('sal').over(window_spec_sum))
df_cumulative.select('ename', 'deptno', 'sal', 'cumulative_salary').show()

+------+------+----+-----------------+
| ename|deptno| sal|cumulative_salary|
+------+------+----+-----------------+
| CLARK|    10|2450|             2450|
|  KING|    10|5000|             7450|
|MILLER|    10|1300|             8750|
| SMITH|    20| 800|              800|
| JONES|    20|2975|             3775|
| SCOTT|    20|3000|             6775|
| ADAMS|    20|1100|             7875|
|  FORD|    20|3000|            10875|
| ALLEN|    30|1600|             1600|
|  WARD|    30|1250|             2850|
|MARTIN|    30|1250|             4100|
| BLAKE|    30|2850|             6950|
|TURNER|    30|1500|             8450|
| JAMES|    30| 950|             9400|
|  JACK|    70|3200|             3200|
+------+------+----+-----------------+



In [221]:
# 부서별 누적 급여
emp_df.groupBy('deptno').agg(
    sum('sal').alias("sal")
).show()


window_spec_sum = Window.partitionBy('deptno').orderBy('empno')
df_cumulative = emp_df.withColumn('cumulative_salary', 
                                sum('sal').over(window_spec_sum))
df_cumulative.select('ename', 'deptno', 'sal', 'cumulative_salary').show()

+------+-----+
|deptno|  sal|
+------+-----+
|    20|10875|
|    10| 8750|
|    70| 3200|
|    30| 9400|
+------+-----+

+------+------+----+-----------------+
| ename|deptno| sal|cumulative_salary|
+------+------+----+-----------------+
| CLARK|    10|2450|             2450|
|  KING|    10|5000|             7450|
|MILLER|    10|1300|             8750|
| SMITH|    20| 800|              800|
| JONES|    20|2975|             3775|
| SCOTT|    20|3000|             6775|
| ADAMS|    20|1100|             7875|
|  FORD|    20|3000|            10875|
| ALLEN|    30|1600|             1600|
|  WARD|    30|1250|             2850|
|MARTIN|    30|1250|             4100|
| BLAKE|    30|2850|             6950|
|TURNER|    30|1500|             8450|
| JAMES|    30| 950|             9400|
|  JACK|    70|3200|             3200|
+------+------+----+-----------------+



In [222]:
# 부서별 평균 급여와 직원 개별 급여 비교?
window_spec_avg = Window.partitionBy('deptno')
df_avg_compare = emp_df.withColumn('dept_avg_salary', 
                        avg('sal').over(window_spec_avg))
df_avg_compare.select('ename', 'deptno', 'sal', 'dept_avg_salary').show()

+------+------+----+------------------+
| ename|deptno| sal|   dept_avg_salary|
+------+------+----+------------------+
| CLARK|    10|2450|2916.6666666666665|
|  KING|    10|5000|2916.6666666666665|
|MILLER|    10|1300|2916.6666666666665|
| SMITH|    20| 800|            2175.0|
| JONES|    20|2975|            2175.0|
| SCOTT|    20|3000|            2175.0|
| ADAMS|    20|1100|            2175.0|
|  FORD|    20|3000|            2175.0|
| ALLEN|    30|1600|1566.6666666666667|
|  WARD|    30|1250|1566.6666666666667|
|MARTIN|    30|1250|1566.6666666666667|
| BLAKE|    30|2850|1566.6666666666667|
|TURNER|    30|1500|1566.6666666666667|
| JAMES|    30| 950|1566.6666666666667|
|  JACK|    70|3200|            3200.0|
+------+------+----+------------------+



In [224]:
emp_df.cube('deptno', 'job').agg(count('*'), sum('sal')).orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    5600|
|    10|     NULL|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [234]:
# 최소, 최대, 평균
emp_df.cube('deptno').agg(count('*'), min('sal'), max('sal'), round(avg('sal'), 2)).orderBy('deptno').show()

+------+--------+--------+--------+------------------+
|deptno|count(1)|min(sal)|max(sal)|round(avg(sal), 2)|
+------+--------+--------+--------+------------------+
|  NULL|      15|     800|    5000|           2148.33|
|    10|       3|    1300|    5000|           2916.67|
|    20|       5|     800|    3000|            2175.0|
|    30|       6|     950|    2850|           1566.67|
|    70|       1|    3200|    3200|            3200.0|
+------+--------+--------+--------+------------------+



In [236]:
# 최소, 최대, 평균
emp_df.cube('job').agg(count('*'), min('sal'), max('sal'), round(avg('sal'), 2)).orderBy('job').show()

+---------+--------+--------+--------+------------------+
|      job|count(1)|min(sal)|max(sal)|round(avg(sal), 2)|
+---------+--------+--------+--------+------------------+
|     NULL|      15|     800|    5000|           2148.33|
|  ANALYST|       2|    3000|    3000|            3000.0|
|    CLERK|       5|     800|    3200|            1470.0|
|  MANAGER|       3|    2450|    2975|           2758.33|
|PRESIDENT|       1|    5000|    5000|            5000.0|
| SALESMAN|       4|    1250|    1600|            1400.0|
+---------+--------+--------+--------+------------------+



In [240]:
join_df = emp_df.join(dept_df, on='deptno', how='inner')
join_df.show()

+------+-----+------+---------+----+----------+----+----+----------+--------+
|deptno|empno| ename|      job| mgr|  hiredate| sal|comm|     dname|     loc|
+------+-----+------+---------+----+----------+----+----+----------+--------+
|    20| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|  RESEARCH|  DALLAS|
|    30| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|     SALES| CHICAGO|
|    30| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|     SALES| CHICAGO|
|    20| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|  RESEARCH|  DALLAS|
|    30| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|     SALES| CHICAGO|
|    30| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|     SALES| CHICAGO|
|    10| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|ACCOUNTING|NEW YORK|
|    20| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|  RESEARCH|  DALLAS|
|    10| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|ACCOUNTING|NEW YORK|
|    30| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|     SA

In [241]:
spark.stop()